# Getting Structured Outputs from Llama3 using Ollama 

In [ ]:
!pip install -U ollama pydantic

In [1]:
from ollama import chat
from pydantic import BaseModel
from typing import Optional, List

class Task(BaseModel):
    title: str
    description: str
    priority: str  # "High", "Medium", "Low"
    estimated_hours: float
    dependencies: List[str] = []
    status: str = "Not Started"  # "Not Started", "In Progress", "Completed"
    assigned_to: Optional[str] = None

response = chat(
    messages=[
        {
            'role': 'user',
            'content': 'Create a task for implementing a new feature in our project management software that allows users to track time spent on tasks. \
            Include dependencies and make it realistic.',
        }
    ],
    model='mistral-small3.2',
    format=Task.model_json_schema(),
)

task = Task.model_validate_json(response['message']['content'])
task

Task(title='Implement Time Tracking Feature for Tasks', description='As a project management software, we need to allow users to track the time they spend on individual tasks. This feature will help with better time management, reporting, and billing.', priority='High', estimated_hours=40.0, dependencies=[], status='Not Started', assigned_to=None)

In [2]:
print(f"Task: {task.title}")
print(f"Priority: {task.priority}")
print(f"Estimated Hours: {task.estimated_hours}")
print(f"Dependencies: {', '.join(task.dependencies)}")
print(f"Status: {task.status}")

Task: Implement Time Tracking Feature for Tasks
Priority: High
Estimated Hours: 40.0
Dependencies: 
Status: Not Started


In [3]:
class EmailToolInput(BaseModel):
    email_destination: str
    email_contents: str


prompt = """
Write an email to my boss: boss_of_lucas@gmail.com, telling him that I quit to pursue bird watching.
"""

email_reply_input = chat(
    model='mistral-small3.2',
    messages=[
        {
            'role': 'user',
            'content': prompt,
        }
    ],
    format=EmailToolInput.model_json_schema(),
)

email_reply_input

{'model': 'mistral-small3.2',
 'created_at': '2025-07-30T15:53:05.084035Z',
 'message': {'role': 'assistant',
  'content': '{ "email_destination": "boss_of_lucas@gmail.com", "email_contents": "Subject: Resignation Notice - Pursuing a New Passion\\n\\nDear [Boss\'s Name],\\n\\nI hope this email finds you well. I am writing to formally resign from my position at [Company Name], effective [last working day, typically two weeks from the date of the email].\\n\\nThis decision was not made lightly, but I have decided to pursue a long-held passion of mine: bird watching. I have always found great joy and fulfillment in observing and learning about birds, and I am excited to dedicate more time to this hobby and potentially turn it into a new career path.\\n\\nI am grateful for the opportunities and experiences I have had during my time here. I have learned a great deal and have truly appreciated working with such a talented and supportive team. Please know that I am committed to making the tra

In [5]:
email_reply_input_json = EmailToolInput.model_validate_json(email_reply_input['message']['content'])
print(email_reply_input_json.email_destination)

print(email_reply_input_json.email_contents)

boss_of_lucas@gmail.com
Subject: Resignation Notice - Pursuing a New Passion

Dear [Boss's Name],

I hope this email finds you well. I am writing to formally resign from my position at [Company Name], effective [last working day, typically two weeks from the date of the email].

This decision was not made lightly, but I have decided to pursue a long-held passion of mine: bird watching. I have always found great joy and fulfillment in observing and learning about birds, and I am excited to dedicate more time to this hobby and potentially turn it into a new career path.

I am grateful for the opportunities and experiences I have had during my time here. I have learned a great deal and have truly appreciated working with such a talented and supportive team. Please know that I am committed to making the transition as smooth as possible. I am more than willing to assist in training a replacement or documenting my responsibilities to ensure a seamless handover.

Thank you again for the oppor

In [6]:
def send_email(email_destination: str, email_contents: str):
    print("Pretending I am sending an email")
    print(f"Sending email to {email_destination} with contents: {email_contents}")

send_email(email_reply_input_json.email_destination, email_reply_input_json.email_contents)

Pretending I am sending an email
Sending email to boss_of_lucas@gmail.com with contents: Subject: Resignation Notice - Pursuing a New Passion

Dear [Boss's Name],

I hope this email finds you well. I am writing to formally resign from my position at [Company Name], effective [last working day, typically two weeks from the date of the email].

This decision was not made lightly, but I have decided to pursue a long-held passion of mine: bird watching. I have always found great joy and fulfillment in observing and learning about birds, and I am excited to dedicate more time to this hobby and potentially turn it into a new career path.

I am grateful for the opportunities and experiences I have had during my time here. I have learned a great deal and have truly appreciated working with such a talented and supportive team. Please know that I am committed to making the transition as smooth as possible. I am more than willing to assist in training a replacement or documenting my responsibilit

We can turn this structured output functionality into a function calling pipeline.